In [ ]:
import numpy as np
import sep

## Additional setup for reading the test image and displaying plots

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline

rcParams['figure.figsize'] = [10.0,8.0]

## Open the FITS file

In [ ]:
fname = "image.fits"
hdu_list = fits.open(fname)
hdu_list.info()

## Read in the data

In [ ]:
data = hdu_list[0].data

## Read image into standard 2-d numpy array

In [ ]:
print(type(data))
print(data.shape)

In [ ]:
image_header = hdu_list[0].header


In [ ]:
hdu_list.close()

## Let's show the data

In [ ]:
m = np.mean(data)
s = np.std(data)
fig1 = plt.figure(figsize=(10.0,8.0))
plt.imshow(data, interpolation = 'nearest', cmap='gray',vmin=m-s,vmax=m+s,origin='lower')
plt.colorbar()
fig1.savefig('data.png')

## Background subtraction

In [ ]:
#measure a spatially varying background on the image
bkg = sep.Background(data)

In [ ]:
#get a "global" mean and noise of the image background
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
#evaluate background as 2D array, same size as original image
bkg_image = bkg.back()
# bkg_image = np.array(bkg) #equivalent to above

In [ ]:
#show the background
fig2 = plt.figure(figsize=(10.0,8.0))
plt.imshow(bkg_image, interpolation='nearest',cmap='gray',origin='lower')
plt.colorbar();
fig2.savefig('Background.png')

In [ ]:
# evaluate the background noise as 2D array, same size as original image
bkg_rms = bkg.rms()

In [ ]:
# show the background noise
fig3 = plt.figure(figsize=(10.0,8.0))
plt.imshow(bkg_rms, interpolation='nearest',cmap='gray',origin='lower')
plt.colorbar();
fig3.savefig('Background_noise.png')

In [ ]:
#subtract the background
data_sub = data - bkg

## Object Detection

In [ ]:
objects = sep.extract(data_sub,1.5,err=bkg.globalrms)

In [ ]:
#how many objects were detected
len(objects)

In [ ]:
from matplotlib.patches import Ellipse
#plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation = 'nearest', cmap = 'gray', vmin=m-s, vmax=m+s,origin='lower')
#plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i],objects['y'][i]), width = 6*objects['a'][i],height=6*objects['b'][i],angle=objects['theta'][i]*180.0/np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
fig.savefig('sources_found_f105w.png')

In [ ]:
#available fields
objects.dtype.names

## Aperture photometry

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'],objects['y'],3.0, err=bkg.globalrms,gain=1.0)

In [ ]:
#show the first 10 objects results
for i in range(10):
    print("object{:d}: flux = {:f}+/- {:f}".format(i, flux[i],fluxerr[i]))